In [1]:
# Step 1: Import Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import LabelEncoder

from sklearn.model_selection import TimeSeriesSplit

In [39]:


# Step 2: Load Data
data = pd.read_csv('./datasets/processed_sales_df.csv')
data.head()
data['date_ordered'] = pd.to_datetime(data['date_ordered'])



In [43]:
# # Step 5: One-Hot Encoding for Categorical Data (Product)
# # data = pd.get_dummies(data, columns=['bike_name'], drop_first=True)
# # X = data[['product_encoded', 'year', 'month', 'day', 'day_of_week', 'quantity_lag1', 'rolling_avg_7']]
# X = data.drop(columns=['quantity','bike_type','date_ordered'])
# print(X)
# y = data['quantity']
# # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# tscv = TimeSeriesSplit(n_splits=5)

# for train_index, test_index in tscv.split(X):
#     X_train, X_test = X[train_index], X[test_index]
#     y_train, y_test = y[train_index], y[test_index]


# # Step 7: Model Training (Random Forest)
# model = RandomForestRegressor(n_estimators=100, random_state=42)
# model.fit(X_train, y_train)

# # Step 8: Model Evaluation
# y_pred = model.predict(X_test)
# mae = mean_absolute_error(y_test, y_pred)
# print(f'Mean Absolute Error: {mae}')

from sklearn.model_selection import TimeSeriesSplit
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression
print((data.drop(columns=['bike_name','bike_type']).columns).tolist())
# Define your categorical and numerical features

X = data
y = data['quantity']

categorical_features = ['bike_name']
numerical_features = ['quantity', 'year', 'month', 'day', 'day_of_week', 'quantity_lag1', 'rolling_avg_7']
# Define transformers
categorical_transformer = OneHotEncoder(handle_unknown='ignore')
numerical_transformer = 'passthrough'  # Or use StandardScaler(), etc.

# Create preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical_features),
        ('num', numerical_transformer, numerical_features)
    ])

# Create pipeline
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', LinearRegression())
])

# Time series cross-validation
tscv = TimeSeriesSplit(n_splits=5)

for train_index, test_index in tscv.split(X):
    # Use .iloc to select rows by integer position
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    # Fit and predict
    pipeline.fit(X_train, y_train)
    print(data.head())
    y_pred = pipeline.predict(X_test)
    
    # Evaluate the model
    # e.g., calculate MAE, MSE, etc.



['date_ordered', 'quantity', 'year', 'month', 'day', 'day_of_week', 'quantity_lag1', 'rolling_avg_7']
        bike_name  bike_type date_ordered  quantity  year  month  day  \
0   Speedster Pro  Road Bike   2018-01-06         3  2018      1    6   
1   Speedster Pro  Road Bike   2018-01-07         2  2018      1    7   
2   Speedster Pro  Road Bike   2018-01-07         2  2018      1    7   
3   Speedster Pro  Road Bike   2018-01-07         2  2018      1    7   
4   Speedster Pro  Road Bike   2018-01-08         3  2018      1    8   

   day_of_week  quantity_lag1  rolling_avg_7  
0            5            2.0       2.714286  
1            6            3.0       2.714286  
2            6            2.0       2.428571  
3            6            2.0       2.285714  
4            0            2.0       2.285714  
        bike_name  bike_type date_ordered  quantity  year  month  day  \
0   Speedster Pro  Road Bike   2018-01-06         3  2018      1    6   
1   Speedster Pro  Road Bike   

In [44]:
from sklearn.metrics import r2_score

r_squared = r2_score(y_test, y_pred)
print(f'R^2 Score: {r_squared}')

R^2 Score: 0.9999998956044827


In [45]:
# Step 9: Future Prediction for all products (next 7 days example)
future_dates = pd.date_range(start=data['date_ordered'].max(), periods=7, freq='D')
future_data = pd.DataFrame({'date_ordered': future_dates})

# Prepare features for each product
all_future_preds = {}
for product in data['bike_name'].unique():
    # Create a copy for each product's prediction
    future_data_copy = future_data.copy()
    
    # Set the time-based features
    future_data_copy['year'] = future_data_copy['date_ordered'].dt.year
    future_data_copy['month'] = future_data_copy['date_ordered'].dt.month
    future_data_copy['day'] = future_data_copy['date_ordered'].dt.day
    future_data_copy['day_of_week'] = future_data_copy['date_ordered'].dt.dayofweek

    # Set lag and rolling average for the product based on recent data
    last_known_quantity = data[data['bike_name'] == product]['quantity'].iloc[-1]
    last_rolling_avg = data[data['bike_name'] == product]['rolling_avg_7'].iloc[-1]
    
    future_data_copy['quantity_lag1'] = last_known_quantity
    future_data_copy['rolling_avg_7'] = last_rolling_avg
    
    # Encode the product
    future_data_copy['product_encoded'] = le.transform([product])[0]

    # Predict the quantity for the next 7 days for this product
    future_preds = model.predict(future_data_copy[['product_encoded', 'year', 'month', 'day', 'day_of_week', 'quantity_lag1', 'rolling_avg_7']])
    
    # Store the predictions for this product
    all_future_preds[product] = future_preds


# Print future predictions for each product
for product, preds in all_future_preds.items():
    print(f"Future Predictions for {product}: {preds}")

# future_dates = pd.date_range(start=data['date_ordered'].max() + pd.Timedelta(days=1), periods=7, freq='D')

# # Step 2: Prepare future data for each product
# all_future_preds = {}

# # Get the list of unique products
# products = data['bike_name'].unique()

# for product in products:
#     # Create a DataFrame for future dates
#     future_data = pd.DataFrame({'date_ordered': future_dates})

#     # Add time-based features
#     future_data['year'] = future_data['date_ordered'].dt.year
#     future_data['month'] = future_data['date_ordered'].dt.month
#     future_data['day'] = future_data['date_ordered'].dt.day
#     future_data['day_of_week'] = future_data['date_ordered'].dt.dayofweek

#     # Get the most recent data for the product
#     product_data = data[data['bike_name'] == product]

#     # Check if product data is available
#     if not product_data.empty:
#         # Set lag features based on the last known values
#         last_known_quantity = product_data['quantity'].iloc[-1]
#         last_rolling_avg = product_data['rolling_avg_7'].iloc[-1]
#     else:
#         # Set default values or handle accordingly
#         last_known_quantity = 0
#         last_rolling_avg = 0

#     # Add lag features to future data
#     future_data['quantity_lag1'] = last_known_quantity
#     future_data['rolling_avg_7'] = last_rolling_avg

#     # Add the product name
#     future_data['bike_name'] = product

#     # Define the feature columns
#     feature_columns = ['bike_name', 'year', 'month', 'day', 'day_of_week', 'quantity_lag1', 'rolling_avg_7']

#     # Select the features
#     X_future = future_data[feature_columns]

#     # Ensure data types are consistent
#     X_future['bike_name'] = X_future['bike_name'].astype(str)

#     # Predict using the pipeline
#     future_preds = pipeline.predict(X_future)

#     # Store predictions
#     all_future_preds[product] = future_preds

# # Step 3: Print future predictions for each product
# for product, preds in all_future_preds.items():
#     print(f"Future Predictions for {product}: {preds}")

NameError: name 'le' is not defined

In [46]:

# Future predictions
future_dates = pd.date_range(start=data['date_ordered'].max() + pd.Timedelta(days=1), periods=7, freq='D')
all_future_preds = {}
products = data['bike_name'].unique()

for product in products:
    future_data = pd.DataFrame({'date_ordered': future_dates})
    future_data['year'] = future_data['date_ordered'].dt.year
    future_data['month'] = future_data['date_ordered'].dt.month
    future_data['day'] = future_data['date_ordered'].dt.day
    future_data['day_of_week'] = future_data['date_ordered'].dt.dayofweek
    product_data = data[data['bike_name'] == product]
    if not product_data.empty:
        last_known_quantity = product_data['quantity'].iloc[-1]
        last_rolling_avg = product_data['rolling_avg_7'].iloc[-1]
    else:
        last_known_quantity = 0
        last_rolling_avg = 0
    quantity_lag1 = last_known_quantity
    rolling_avg_7 = last_rolling_avg
    predictions = []
    for i in range(len(future_dates)):
        row = {
            'bike_name': product,
            'year': future_data.iloc[i]['year'],
            'month': future_data.iloc[i]['month'],
            'day': future_data.iloc[i]['day'],
            'day_of_week': future_data.iloc[i]['day_of_week'],
            'quantity_lag1': quantity_lag1,
            'rolling_avg_7': rolling_avg_7
        }
        X_future = pd.DataFrame([row])
        pred = pipeline.predict(X_future)[0]
        predictions.append(pred)
        quantity_lag1 = pred
        # Update rolling_avg_7 if needed
    all_future_preds[product] = predictions

# Print predictions
for product, preds in all_future_preds.items():
    print(f"Future Predictions for {product}:")
    for date, pred in zip(future_dates, preds):
        print(f"Date: {date.date()}, Predicted Quantity: {pred}")
    print("\n")

ValueError: columns are missing: {'quantity'}

In [13]:
all_future_preds_df = pd.DataFrame(all_future_preds)
all_future_preds_df.to_csv('./datasets/sales_predictions.csv', index=False)
